In [1]:
import sys
import os

# from src.puzzle_creators import Direction

SCRIPT_DIR = os.path.dirname(os.path.abspath(os.getcwd()+"/notebooks"))
sys.path.append(os.path.dirname(SCRIPT_DIR))


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import os
from functools import reduce
from pathlib import Path
from src.puzzle_creators import statistics

In [3]:
def n_interior_points(example_name):
    ex = example_name.split("\\")[-1]
    values = ex.split("-")
    return eval(values[-2])

def n_convex_hull(example_name):
    ex = example_name.split("\\")[-1]
    values = ex.split("-")
    return eval(values[1])


examples_path = os.path.join(Path().resolve().parent,"data","puzzles")
examples = list(filter(lambda exa: n_convex_hull(exa)+n_interior_points(exa) == 5,
                 [ex for ex in glob.glob(f"{examples_path}/convex_hull*")]))
examples = sorted(examples,key=lambda ex: n_convex_hull(ex)+n_interior_points(ex)*1.01)

df_examples = pd.DataFrame({
    "name": list(map(lambda ex: ex.split("\\")[-1],examples)),
    "n_convex_hull":list(map(n_convex_hull,examples)),
    "n_interior":list(map(n_interior_points,examples)),
    # "n_puzzles":[len(glob.glob(f"{sample_dir}/results/*.csv")) for sample_dir in examples],
    "example_path":examples
})

df_examples.head()

,name,n_convex_hull,n_interior,example_path
0,convex_hull-5-int-0-107,5,0,C:\Users\yaniv\Desktop\MSCBenGurion\iCVL\rgons...
1,convex_hull-5-int-0-1826,5,0,C:\Users\yaniv\Desktop\MSCBenGurion\iCVL\rgons...
2,convex_hull-5-int-0-2039,5,0,C:\Users\yaniv\Desktop\MSCBenGurion\iCVL\rgons...
3,convex_hull-5-int-0-3371,5,0,C:\Users\yaniv\Desktop\MSCBenGurion\iCVL\rgons...
4,convex_hull-5-int-0-367,5,0,C:\Users\yaniv\Desktop\MSCBenGurion\iCVL\rgons...


#Build df_all_puzzles
#The columns should be sample_name (example_name), puzzle_name, n_3_polygons,n_4_polygons...

In [43]:
counts = []
samples_names = []
puzzles_names = []
dfs = []
for sample_dir in examples:
    for puzzle_csv in glob.glob(f"{sample_dir}/results/*.csv"):
        samples_names.append(sample_dir.split("\\")[-1])
        puzzles_names.append(puzzle_csv.split("\\")[-1])
        df_puzzle_polygons = pd.read_csv(puzzle_csv,index_col=False)
        df_puzzle_polygons = df_puzzle_polygons.groupby("id").size().reset_index(name="polygon_type")
        # print(df_puzzle_polygons.head(20))
        # print(df_puzzle_polygons.columns)
        df = pd.DataFrame(df_puzzle_polygons.value_counts(["polygon_type"])).reset_index()
        df.columns = ["polygon_type","count"]
        # print(df.head())
        # print(df.columns)
        df = df.set_index("polygon_type")
        df = df.T#.reset_index(drop=True)
        df.columns = [f"polygon_{col}" for col in df.columns]
        df = df.reset_index(drop=True)
        dfs.append(df)
        # print(df.head())
        # print(df.columns)
        
all_puzzles = pd.concat(dfs).fillna(0)
all_puzzles["name"] = puzzles_names
all_puzzles["sample"] = samples_names
all_puzzles = all_puzzles.reset_index(drop=True)
cols = list(all_puzzles.columns)
all_puzzles = all_puzzles[cols[-2:]+cols[:-2]]
print(all_puzzles.head(5))
print(len(all_puzzles))



             name                   sample  polygon_3  polygon_4  polygon_5
0  1-9_1-3_s_.csv  convex_hull-5-int-0-107        3.0        0.0        0.0
1    1-9_2-3_.csv  convex_hull-5-int-0-107        3.0        0.0        0.0
2    1-9_3-3_.csv  convex_hull-5-int-0-107        1.0        1.0        0.0
3      2-9_s_.csv  convex_hull-5-int-0-107        1.0        1.0        0.0
4        3-9_.csv  convex_hull-5-int-0-107        0.0        0.0        1.0


350

#For each example calculate the expected value and variance